### How Many Indian Politician Accounts Have Been Hacked?

In [1]:
import os
import json
import re
import requests
import time
import pandas as pd

df_list = []

for filename in os.listdir("daughters/data/"):
    print(filename)
    with open(os.path.join("daughters/data/", filename), "r") as f:
            data = json.load(f)
    df = pd.DataFrame(data['membersDtoList'])
    df['ls'] = filename[3:-5]
    df_list.append(df)

df = pd.concat(df_list)

ls_17.json
ls_16.json
ls_13.json
ls_14.json
ls_15.json
ls_12.json


In [2]:
df_emails = df['email'].explode().reset_index()
df_emails = df_emails['email'].str.split(r'</br>').explode().reset_index()
df_emails['email'] = df_emails['email'].str.strip()
df_emails['email_fix'] = df_emails['email'].str.replace('\[at\]', '@', regex=True).str.replace('\[dot\]', '.', regex=True)
df_emails['email_fix'] = df_emails['email_fix'].str.extract(r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})')

In [3]:
df = df.reset_index()

In [4]:
df_joined = df.join(df_emails,on='index', how='outer', lsuffix='_left', rsuffix='_right')
df_joined = df_joined.reset_index()

In [5]:
df_small = df_joined[df_joined['email_fix'].notnull()]
df_small['email_fix'].isna().sum()

0

In [ ]:
# HIBP
url = "https://haveibeenpwned.com/api/v3/breachedaccount/<account>"
hibp_api_key = 'cfd19b48bb234115a2a509d1f2b5fa11'
payload={}
headers = {
  'hibp-api-key': str(hibp_api_key),
  'format': 'application/json',
  'timeout': '2.5',
  'HIBP': str(hibp_api_key),
}

In [ ]:
for index, row in df_small.iterrows():
    email = row['email_fix']
    url = f"https://haveibeenpwned.com/api/v3/breachedaccount/{email}"
    response = requests.request("GET", url, headers=headers, data=payload)
    time.sleep(6)
    print(index)
    with open(f"pwned/{email}.json", "wb") as f:
        f.write(response.content)